# Project(lateset)

## General set-up

In [0]:
#@title File upload
from google.colab import files
files.upload()

In [0]:
#@title GPU detection
# GPU detection
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
#@title Dependencies
import sys
import time
import numpy as np
import tensorflow as tf
from sklearn import svm
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, average_precision_score, precision_recall_curve, precision_recall_fscore_support, roc_auc_score

## OC-SVM

In [0]:
#@title OC-SVM set-up
def OCSVM_preprocess(anomaly_ratio=0.2, anom_label=None):
    rng = np.random.RandomState(42)

    data = np.load('mnist.npz')
    full_x_data = np.concatenate([data['x_train'], data['x_test'], data['x_valid']], axis=0) # [70k x 728]
    full_y_data = np.concatenate([data['y_train'], data['y_test'], data['y_valid']], axis=0) # [70k]
    if anom_label == 'notmnist':
        not_mnist = np.load('not_mnist.npz')
        full_notmnist = not_mnist['data'].reshape(-1,784)
        rnd_inds_norm = rng.permutation(full_x_data.shape[0])
        rnd_inds_anom = rng.permutation(full_notmnist.shape[0])

        x_train = full_x_data[rnd_inds_norm][:20000]
        x_test_norm = full_x_data[rnd_inds_norm][20000:25000]
        x_test_anom = full_notmnist[rnd_inds_anom][:1250]

        x_test = np.concatenate((x_test_norm,x_test_anom),axis=0)
        y_test = np.concatenate((np.ones((x_test_norm.shape[0])),
                             -1*np.ones((x_test_anom.shape[0]))), axis=0)

        rand_test = rng.permutation(x_test.shape[0])
        x_test = x_test[rand_test]
        y_test = y_test[rand_test]

    elif anom_label is not None:
        anom_inds = np.isin(full_y_data,anom_label)
        norm_inds = np.invert(anom_inds)

        norm_x_data = full_x_data[norm_inds]
        anom_x_data = full_x_data[anom_inds]

        rnd_inds_norm = rng.permutation(norm_x_data.shape[0])
        rnd_inds_anom = rng.permutation(anom_x_data.shape[0])
        x_train = norm_x_data[rnd_inds_norm][:20000]
        x_test_norm = norm_x_data[rnd_inds_norm][20000:25000]
        x_test_anom = anom_x_data[rnd_inds_anom][:1250]

        x_test = np.concatenate((x_test_norm,x_test_anom),axis=0)
        y_test = np.concatenate((np.ones((x_test_norm.shape[0])),
                             -1*np.ones((x_test_anom.shape[0]))), axis=0)

        rand_test = rng.permutation(x_test.shape[0])
        x_test = x_test[rand_test]
        y_test = y_test[rand_test]
        
#         x_train = x_train * 2. - 1.
#         x_test = x_test * 2. - 1.
    
    return x_train, x_test, y_test

def build_svm(oc=False,_nu=0.5,_gamma='auto',maxiter=-1):
    if oc == False:
        clf = svm.NuSVC(nu=_nu,gamma=_gamma,max_iter=maxiter)
    if oc == True:
        clf = svm.OneClassSVM(nu=_nu,gamma=_gamma,max_iter=maxiter)

    return clf

def plot_25(image):
    '''
        Plot 25 images generated by GEN every 5 epochs
        Args: ?*[25 x 28 x 28 x 1] (? is the integer part of nb_epochs/5)
    '''
    plots = image.reshape(-1,28,28) # [25 x 28 x 28]
    ind = 0
    plt.show()
    while ind < 25:
        plt.subplot(5,5,ind+1)
        img = plt.imshow(plots[ind,:,:])
        plt.axis('off')
        ind += 1

In [0]:
#@title Set-up contd
nu_=0.01 
gamma_=0.05 
iter_=-1
ex_x_train, ex_x_test, ex_y_test = OCSVM_preprocess(anom_label='notmnist')
print('OC SVM with external dataset')
begin = time.time()
ocsvm = build_svm(oc=True, _nu=nu_, _gamma=gamma_, maxiter=iter_)
ocsvm.fit(ex_x_train.reshape(-1,784))
predy = ocsvm.predict(ex_x_test.reshape(-1,784))
prec, reca, f1, _ = metrics.precision_recall_fscore_support(ex_y_test, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(ex_y_test, predy)
print('anomalous digit: ', i)
i += 1
print('nu: ', nu_)
print('width: ', gamma_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('time: ', time.time()-begin)

oc_train_x_one = []
oc_test_x_one = []
oc_test_y_one = []
for i in range(10):
    oc_train_x ,oc_test_x, oc_test_y = OCSVM_preprocess(anom_label=[i])
    oc_train_x_one.append(oc_train_x)
    oc_test_x_one.append(oc_test_x)
    oc_test_y_one.append(oc_test_y)
    
oc_train_x_two = []
oc_test_x_two = []
oc_test_y_two = []
for i in [[1,7],[3,5],[4,9]]:
    oc_train_x ,oc_test_x, oc_test_y = OCSVM_preprocess(anom_label=[i])
    oc_train_x_two.append(oc_train_x)
    oc_test_x_two.append(oc_test_x)
    oc_test_y_two.append(oc_test_y)

In [0]:
#@title One digit
print('OC SVM with one digit')
i = 0
for trainx, testx, testy in zip(oc_train_x_one,oc_test_x_one,oc_test_y_one):
    begin = time.time()
    ocsvm = build_svm(oc=True, _nu=nu_, _gamma=gamma_, maxiter=iter_)
    ocsvm.fit(trainx.reshape(-1,784))
    predy = ocsvm.predict(testx.reshape(-1,784))
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy, predy)
    print('anomalous digit: ', i)
    i += 1
    print('nu: ', nu_)
    print('width: ', gamma_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('time: ', time.time()-begin)
    

In [0]:
#@title Two digits
print('OC SVM with two digits')
i = 0
for trainx, testx, testy in zip(oc_train_x_two,oc_test_x_two,oc_test_y_two):
    begin = time.time()
    ocsvm = build_svm(oc=True, _nu=nu_, _gamma=gamma_, maxiter=iter_)
    ocsvm.fit(trainx.reshape(-1,784))
    predy = ocsvm.predict(testx.reshape(-1,784))
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy, predy)
    print('anomalous set: no.%d'%i)
    i += 1
    print('nu: ', nu_)
    print('width: ', gamma_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('time: ', time.time()-begin)

## GAN set-up

In [0]:
#@title Parameters
class hyper_params():
    '''
    Set and display all hyper parameters
    Args:
        {gen,dis}_lr: real number between 0 and 1
        label: interger from [0,9]
        noise: True or False
        nt: 'conv' or 'mlp'
        all others: positive integer
    '''
    def __init__(self, gen_lr, dis_lr, nb_epochs, latent_dim, batch_size, rand_seed, dis_inter_layer_dim, label, noise, nl, nt, sne):
        self._gen_lr = gen_lr
        self._dis_lr = dis_lr
        self._nb_epochs = nb_epochs
        self._z_dim = latent_dim
        self._bs = batch_size
        self._rs = rand_seed
        self._dis_ild = dis_inter_layer_dim
        self._lab = label
        self._noise = noise
        self._nl = nl
        self._nt = nt
        self._sne = sne
        
    def gen_learning_rate(self):
        return self._gen_lr
    
    def dis_learning_rate(self):
        return self._dis_lr
    
    def nb_epochs(self):
        return self._nb_epochs
    
    def latent_dim(self):
        return self._z_dim
    
    def batch_size(self):
        return self._bs
    
    def random_seed(self):
        return self._rs
    
    def dis_inter_layer_dim(self):
        return self._dis_ild 
    
    def label(self):
        return self._lab
    
    def noise(self):
        return self._noise
    
    def noise_level(self):
        return self._nl
    
    def network_type(self):
        return self._nt
    
    def size_neg_exp(self):
        return self._sne
    
    def display(self):
        print('###################################')
        print('Training parameters are as follows:')
        print('Network type: ', self._nt)
        print('Number of epochs: ', self._nb_epochs)
        print('Batch size: ', self._bs)
        print('Generator learning rate: ', self._gen_lr)
        print('Discriminator learninig rate: ', self._dis_lr)
        print('Latent dimension: ', self._z_dim)
        print('Anomaly label: ', self._lab)
        print('Discriminator intermediate layer dimension: ', self._dis_ild)
        print('Gaussian noise incorporated in the discriminator: ', self._noise)
        print('Gaussian noise mean: ', self._nl)
        print('Size of negative exmaples generated after every 5 epoch: ', self._sne)
        print('###################################')

In [0]:
#@title Utility Functions

def plot_gen(images):
    '''
        Plot 25 images generated by GEN every 5 epochs
        Args: ?*[25 x 28 x 28 x 1] (? is the integer part of nb_epochs/5)
    '''
    for i in range(len(images)):
        image = images[i]
        plots = image.reshape(-1,28,28) # [25 x 28 x 28]
        ind = 0
        plt.show()
        print('Generating 25 images at the end of epoch %d' % ((i+1)*5))
        while ind < 25:
            plt.subplot(5,5,ind+1)
            plt.imshow(plots[ind,:,:])
            plt.axis('off')
            ind += 1
            
            
def plot_learning_curves(gen_loss, dis_loss):
    print('Losses of GEN and DIS')
    gen = np.array(gen_loss)
    dis = np.array(dis_loss)
    plt.figure()
    plt.plot(gen, 'b')
    plt.plot(dis, 'g')
    plt.legend(['GEN loss', 'DIS loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
def plot_roc_curves(train_roc, test_roc):
    print('Areas under ROC of DIS on train and test')
    tr_roc = np.array(train_roc)
    te_roc = np.array(test_roc)
    plt.figure()
    plt.plot(tr_roc, 'c')
    plt.plot(te_roc, 'r')
    plt.legend(['train AUROC', 'test AUROC'])
    plt.xlabel('Epoch')
    plt.ylabel('Area under ROC')
    
def plot_roc_curves_svm(train_roc, test_roc):
    print('Areas under ROC of DIS on train and test')
    tr_roc = np.array(train_roc)
    te_roc = np.array(test_roc)
    plt.figure()
    plt.plot(tr_roc, 'c')
    plt.plot(te_roc, 'r')
    plt.legend(['train AUROC', 'test AUROC'])
    plt.xlabel('10 Epochs')
    plt.ylabel('Area under ROC')

In [0]:
#@title Set Parameters


###############################
###############################

###### DO NOT FORGET TO CHANGE!

###############################
###############################




paras = hyper_params(gen_lr=1e-4, dis_lr=1e-5, nb_epochs=70, latent_dim=100, batch_size=100,
# paras = hyper_params(gen_lr=1e-5, dis_lr=1e-5, nb_epochs=50, latent_dim=100, batch_size=100,
                     rand_seed=None, dis_inter_layer_dim=1024, label=[4,9], noise=False, nl=0.1, nt='conv', sne=5000)

In [0]:
#@title Preprocessing

rng = np.random.RandomState(paras.random_seed())

ar = 0.25

def get_train(label, centered=True, anom_ratio=ar):
    # training set
    return _get_adapted_dataset("train", label, centered=centered, anom_ratio=anom_ratio)

def get_valid(label, centered=True, anom_ratio=ar):
    # validation set
    return _get_adapted_dataset("valid", label, centered=centered, anom_ratio=anom_ratio)

def get_test(label, centered=True, anom_ratio=ar):
    # testing set
    return _get_adapted_dataset("test", label, centered=centered, anom_ratio=anom_ratio)

def get_shape_input():
    """Get shape of the dataset for MNIST"""
    return (None, 28, 28, 1)


def _get_adapted_dataset(split, label=None, centered=False, anom_ratio=0.25):
    """Gets the adapted dataset for the experiments

    Args :
            split (str): train or test
            label (list or str): 
                  (list):ints in range 0 to 10, the digits
                         which are considered outlier
                  (str): 'external', indicating external datasets are used
                                
            centered (bool): (Default=False) data centered to [-1, 1]
            anom_ratio (float): between 0 and 1, below 0.5 if label is external
    """
    data = np.load('mnist.npz')
    dataset = {}

    full_x_data = np.concatenate([data['x_train'], data['x_test'], data['x_valid']], axis=0) # [70k x 728]
    full_y_data = np.concatenate([data['y_train'], data['y_test'], data['y_valid']], axis=0) # [70k]

    if label == 'external':
        not_mnist = np.load('not_mnist.npz')
        full_notmnist = not_mnist['data'].reshape(-1,784)
        rnd_inds_norm = rng.permutation(full_x_data.shape[0])
        rnd_inds_anom = rng.permutation(full_notmnist.shape[0])

        train_lab = int(full_x_data.shape[0] * 0.8)
        x_train = full_x_data[rnd_inds_norm][:train_lab]
        y_train = full_y_data[rnd_inds_norm][:train_lab]
        if centered:
            x_train = x_train * 2. - 1.
        x_test_norm = full_x_data[rnd_inds_norm][train_lab:train_lab+5000]
        y_test_norm = full_y_data[rnd_inds_norm][train_lab:train_lab+5000]
        x_valid_norm = full_x_data[rnd_inds_norm][train_lab+5000:train_lab+10000]
        y_valid_norm = full_y_data[rnd_inds_norm][train_lab+5000:train_lab+10000]
        x_test_anom = full_notmnist[rnd_inds_anom][:1250]
        x_valid_anom = full_notmnist[rnd_inds_anom][1250:2500]

        x_test = np.concatenate((x_test_norm,x_test_anom),axis=0) # [6250 x 728]
        x_valid = np.concatenate((x_valid_norm,x_valid_anom),axis=0) # [6250 x 728]
        if centered:
            x_test = x_test * 2. - 1.
            x_valid = x_valid * 2. - 1.
        
        # y_test and y_valid need to be changed from (-1,1) to (0,1) to train the networks 
        y_test = np.concatenate((np.ones((x_test_norm.shape[0])),
                             -1*np.ones((x_test_anom.shape[0]))), axis=0)

        y_valid = np.concatenate((np.ones((x_valid_norm.shape[0])),
                             -1*np.ones((x_valid_anom.shape[0]))), axis=0)

        rand_test = rng.permutation(x_test.shape[0])
        x_test = x_test[rand_test]
        y_test = y_test[rand_test]

        rand_valid = rng.permutation(x_valid.shape[0])
        x_valid = x_valid[rand_valid]
        y_valid = y_valid[rand_valid]
        
        x_train = x_train.reshape(-1, 28, 28, 1)
        x_test = x_test.reshape(-1, 28, 28, 1)
        x_valid = x_valid.reshape(-1, 28, 28 ,1)

        if split == 'train':
            return x_train, y_train

        if split == 'test':
            return x_test, y_test

        if split == 'valid':
            return x_valid, y_valid


    anom_inds = np.isin(full_y_data,label)
    norm_inds = np.invert(anom_inds)

    norm_x_data = full_x_data[norm_inds]
    norm_y_data = full_y_data[norm_inds]

    anom_x_data = full_x_data[anom_inds]
    anom_y_data = full_y_data[anom_inds]

    # shuffle the normal data
    rnd_norm_inds = rng.permutation(norm_x_data.shape[0])
    norm_x_data = norm_x_data[rnd_norm_inds]
    norm_y_data = norm_y_data[rnd_norm_inds]

    rnd_anom_inds = rng.permutation(anom_x_data.shape[0])
    anom_x_data = anom_x_data[rnd_anom_inds]
    anom_y_data = anom_y_data[rnd_anom_inds]


    # train index
    train_index = int(norm_x_data.shape[0]*0.8) # 80% of normal data        

    # train: 80% of normal
    train_x_data = norm_x_data[:train_index]
    train_y_data = norm_y_data[:train_index]

    # test: 5000 of normal, 1250 of anomalous
    # valid: 5000 of normal, 1250 of anomalous
    anom_index = int(5000 * anom_ratio) # a*5000
    test_x_data = np.concatenate((norm_x_data[train_index:train_index+5000], anom_x_data[:anom_index]), axis=0) # [(1+a)*5000 x 784]
    test_y_data = np.concatenate((norm_y_data[train_index:train_index+5000], anom_y_data[:anom_index]), axis=0) # [(1+a)*5000]
    valid_x_data = np.concatenate((norm_x_data[train_index+5000:train_index+10000], anom_x_data[anom_index:2*anom_index]), axis=0) # [(1+a)*5000 x 784]
    valid_y_data = np.concatenate((norm_y_data[train_index+5000:train_index+10000], anom_y_data[anom_index:2*anom_index]), axis=0) # [(1+a)*5000]

    # when a = 0.25:
    # testx: [6250 x 784]
    # testy: [6250]
    # validx: [6250 x 784]
    # validy: [6250]
    
    # shuffle the testing data
    rnd_test_inds = rng.permutation(test_x_data.shape[0])
    test_x_data = test_x_data[rnd_test_inds]
    test_y_data = test_y_data[rnd_test_inds]

    rnd_valid_inds = rng.permutation(valid_x_data.shape[0])
    valid_x_data = valid_x_data[rnd_valid_inds]
    valid_y_data = valid_y_data[rnd_valid_inds]


    key_img = 'x_' + split
    key_lbl = 'y_' + split

    if split == 'train':
        # get the training set
        dataset[key_img] = train_x_data.astype(np.float32)
        dataset[key_lbl] = train_y_data.astype(np.float32)

    elif split == 'test':
        # get the testing set
        dataset[key_img] = test_x_data.astype(np.float32)
        dataset[key_lbl] = test_y_data.astype(np.float32)

    elif split == 'valid':
        # get the testing set           
        dataset[key_img] = valid_x_data.astype(np.float32)
        dataset[key_lbl] = valid_y_data.astype(np.float32)
    
    if centered:
        dataset[key_img] = dataset[key_img].astype(np.float32)
        dataset[key_img] = dataset[key_img] * 2. - 1.

    # reshape images to a square to be feed in CNN
    dataset[key_img] = dataset[key_img].reshape(-1, 28, 28, 1)
    
    return (dataset[key_img], dataset[key_lbl])



In [0]:
#@title CNN with batch normalisation and leaky ReLU
# Convolutional Nets (both gen and dis)
# The initialisation kernel is subjected to change

init_kernel = tf.random_normal_initializer(mean=0.0, stddev=0.02)

def generator_conv(z_inp, is_training=False, reuse=False):
    """ Generator architecture in tensorflow
    Generates data from the latent space
    Args:
        z_inp (tensor): [batch_size x 200]
        reuse (bool): sharing variables or not
    Returns:
        (tensor): [batch_size x 28 x 28 x 1]
    """
    with tf.variable_scope('generator', reuse=reuse):
        
        name_net = 'layer_1'
        with tf.variable_scope(name_net):
            net = tf.layers.dense(z_inp,
                                  units=1024,
                                  kernel_initializer=init_kernel,
                                  name='fc')
            net = tf.layers.batch_normalization(net,
                                        training=is_training,
                                        name='batch_normalization')
            net = tf.nn.relu(net, name='relu')

        name_net = 'layer_2'
        with tf.variable_scope(name_net):
            net = tf.layers.dense(net,
                                  units=7*7*128,
                                  kernel_initializer=init_kernel,
                                  name='fc')
            net = tf.layers.batch_normalization(net,
                                        training=is_training,
                                        name='batch_normalization')
            net = tf.nn.relu(net, name='relu')

        net = tf.reshape(net, [-1, 7, 7, 128])

        name_net = 'layer_3'
        with tf.variable_scope(name_net):
            net = tf.layers.conv2d_transpose(net,
                                     filters=64,
                                     kernel_size=4,
                                     strides= 2,
                                     padding='same',
                                     kernel_initializer=init_kernel,
                                     name='conv')
            net = tf.layers.batch_normalization(net,
                                        training=is_training,
                                        name='batch_normalization')
            net = tf.nn.relu(net, name='relu')

        name_net = 'layer_4'
        with tf.variable_scope(name_net):
            net = tf.layers.conv2d_transpose(net,
                                     filters=1,
                                     kernel_size=4,
                                     strides=2,
                                     padding='same',
                                     kernel_initializer=init_kernel,
                                     name='conv')
            net = tf.tanh(net, name='tanh')

        return net

def discriminator_conv(x_inp, is_training=False, reuse=False, noise=False):
    """ Discriminator architecture in tensorflow
    Discriminates between real data and generated data
    Args:
        x_inp (tensor): [batch_size x 28 x 28 x 1]
        reuse (bool): sharing variables or not
    Returns:
        logits (tensor): [batch_size x 1]
        intermediate_layer (tensor): [batch_size x 1024]
    """
    
    # Add N(0,1) noise to the input of the discriminator
    if noise == True:
        x_inp += tf.random_normal(tf.shape(x_inp), mean=0.0, stddev=paras.noise_level())

    
    with tf.variable_scope('discriminator', reuse=reuse):

        name_net = 'layer_1'
        with tf.variable_scope(name_net):
            net = tf.layers.conv2d(x_inp,
                           filters=64,
                           kernel_size=4,
                           strides=2,
                           padding='same',
                           kernel_initializer=init_kernel,
                           name='conv')
            net = leakyReLu(net, 0.1, name='leaky_relu')
        
        # Gaussian noise
#         if noise == True:
#             net += tf.random_normal(tf.shape(net), mean=0.0, stddev=tddev=paras.noise_level())

        name_net = 'layer_2'
        with tf.variable_scope(name_net):
            net = tf.layers.conv2d(net,
                           filters=64,
                           kernel_size=4,
                           strides=2,
                           padding='same',
                           kernel_initializer=init_kernel,
                           name='conv')
            net = tf.layers.batch_normalization(net,
                                        training=is_training,
                                        name='batch_normalization')
            net = leakyReLu(net, 0.1, name='leaky_relu')

        net = tf.reshape(net, [-1, 7 * 7 * 64])
        
        # Gaussian noise
#         if noise == True:
#             net += tf.random_normal(tf.shape(net), mean=0.0, tddev=paras.noise_level())


        name_net = 'layer_3'
        with tf.variable_scope(name_net):
            net = tf.layers.dense(net,
                      units=paras.dis_inter_layer_dim(),
                      kernel_initializer=init_kernel,
                      name='fc')
            net = tf.layers.batch_normalization(net,
                                    training=is_training,
                                    name='batch_normalization')
            net = leakyReLu(net, 0.1, name='leaky_relu')

        intermediate_layer = net
        
        # Gaussian noise
#         if noise == True:
#             net += tf.random_normal(tf.shape(net), mean=0.0, stddev=paras.noise_level())


        name_net = 'layer_4'
        with tf.variable_scope(name_net):
            net = tf.layers.dense(net,
                                  units=1,
                                  kernel_initializer=init_kernel,
                                  name='fc')

        net = tf.squeeze(net)

        return net, intermediate_layer

def leakyReLu(x, alpha=0.1, name=None):
    if name:
        with tf.variable_scope(name):
            return _leakyReLu_impl(x, alpha)
    else:
        return _leakyReLu_impl(x, alpha)

def _leakyReLu_impl(x, alpha):
    return tf.nn.relu(x) - (alpha * tf.nn.relu(-x))

In [0]:
#@title Train and Test

def train_and_test(paras, testx, testy):
    
    """
    Args:
        paras: a hyper_params object
        testx: [4000 x 728] (3000 normal and 1000 anomalous)
        testy: [4000] (1 normal and -1 anomalous, need to change to real 1 and fake 0)
    """
    
    # Placeholders
    input_pl = tf.placeholder(tf.float32, shape=get_shape_input())
    is_training_pl = tf.placeholder(tf.bool, [])
    learning_rate_pl = tf.placeholder(tf.float32, shape=())

    # Data
    '''
        # trainx: [train_size x 28 x 28 x 1]
        # trainy: [train_size]
        # testx: [test_size x 28 x 28 x 1]
        # testy: [test_size]
    '''

    # without centred
    trainx, _ = get_train(paras.label())
    trainx_copy = trainx.copy()
    testx = testx.reshape(-1,28,28,1)


    # unpack parameters
    gen_lr = paras.gen_learning_rate()
    dis_lr = paras.dis_learning_rate()
    batch_size = paras.batch_size()
    latent_dim = paras.latent_dim()

    # set random seed
    rng = np.random.RandomState(paras.random_seed())
    # get the number of batches in each epoch
    nr_batches_train = int(trainx.shape[0] / batch_size)

    # display parameters
    paras.display()

    if paras.network_type() == 'conv':
        gen = generator_conv
        dis = discriminator_conv
        
    if paras.network_type() == 'mlp':
        gen = generator_mlp
        dis = discriminator_mlp

    # Sample noise from standard normal distribution, i.e. prior: p_z = N(0,1)
    # random_z: [100 x 200]
    random_z = tf.random_normal([batch_size, latent_dim], mean=0.0, stddev=1.0)

    # Generate images with generator
    # generator: [100 x 28 x 28 x 1]
    generator = gen(random_z, is_training=is_training_pl)

    # Pass real and fake images into discriminator separately
    # real_d: [100]
    # inter_layer_real: [100 x 1024]
    real_d, inter_layer_real = dis(input_pl, is_training=is_training_pl, noise=paras.noise()) # N(0,1) noise
  
    # fake_d: [100]
    # inter_layer_fake: [100 x 1024]
    fake_d, inter_layer_fake = dis(generator, is_training=is_training_pl, reuse=True, noise=paras.noise()) # N(0,1) noise


    # Calculate seperate losses for discriminator with real and fake images
    # sigmoid_cross_entropy will take mean (with weight = 1) automatically so {real, fake}_discriminator_loss are scalars
    real_discriminator_loss = tf.losses.sigmoid_cross_entropy(tf.constant(1, shape=[batch_size]), real_d, scope='real_discriminator_loss')
    fake_discriminator_loss = tf.losses.sigmoid_cross_entropy(tf.constant(0, shape=[batch_size]), fake_d, scope='fake_discriminator_loss')
    # discriminator loss (to be minimise wrt to dis's paras)
    discriminator_loss = real_discriminator_loss + fake_discriminator_loss
    # generator loss (to be minimise wrt to gen's paras)
    generator_loss = tf.losses.sigmoid_cross_entropy(tf.constant(1, shape=[batch_size]), fake_d, scope='generator_loss')
    
    # According to GAN 2014 by I.Goodfellow et.al. the training procedure should be k steps discriminator then 1 step generator
    # G must not be trained too much without updating D otherwise will have mode collapse


    with tf.name_scope('optimizers'):
        # control op dependencies for batch norm and trainable variables
        # All variables
        dvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
        gvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

        # All operations (for batch normalisation)
        update_ops_gen = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='generator')
        update_ops_dis = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='discriminator')
        
        # beta1: momentum decay
        # USE DIFFERENT LEARNING RATES FOR DIS AND GEN
        # dis: dis_lr
        optimizer_dis = tf.train.AdamOptimizer(learning_rate=learning_rate_pl, beta1=0.5, name='dis_optimizer')
        # gen: gen_lr
        optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate_pl, beta1=0.5, name='gen_optimizer')

            
    
    
    
        # run all the generator's ops before optimising gen
        with tf.control_dependencies(update_ops_gen): # attached op for moving average batch norm
            gen_op = optimizer_gen.minimize(generator_loss, var_list=gvars)

        # run all the discriminator's ops before optimising dis    
        with tf.control_dependencies(update_ops_dis):
            dis_op = optimizer_dis.minimize(discriminator_loss, var_list=dvars)
            
        '''
            NB: FIRST UPDATE DIS THEN GEN IN TRAINING
        '''
    true_train_label = tf.squeeze(tf.concat([tf.constant(1, shape=[batch_size]), tf.constant(0, shape=[batch_size])], axis=0)) # [(100+100)]
    pred_train_label = tf.squeeze(tf.concat([tf.sigmoid(real_d), tf.sigmoid(fake_d)], axis=0)) # [(50+50)]
    pred_test_label = tf.squeeze(tf.sigmoid(real_d)) # [5000]



    with tf.Session() as sess:
        # Global Varaibles contain all Model variables, which then contain all Trainable variables
        sess.run(tf.global_variables_initializer())
        
        # initialise batch and epoch
        train_batch = 0 
        epoch = 0
        # initialise trainging log
        train_loss_dis_log = []
        train_loss_gen_log = []
        train_roc_log = []
        test_roc_log = []
        gen_exp_log = []
        to_plot = []

        while epoch < paras.nb_epochs():

            begin = time.time() # recording time
            trainx = trainx[rng.permutation(trainx.shape[0])]  # shuffle training set (to be feed into dis)
            trainx_copy = trainx_copy[rng.permutation(trainx.shape[0])] # shuffle training set (to be feed into gen)

            # train_loss_dis, train_loss_gen = [0, 0]
            train_loss_dis, train_loss_gen, train_roc = [0, 0, 0]
            
            
            ###### Training ######
            for t in range(nr_batches_train):

                # construct randomly permuted minibatches
                ran_from = t * batch_size
                ran_to = (t + 1) * batch_size

                ### Discriminator Training ###
                feed_dict = {input_pl: trainx[ran_from:ran_to],
                             is_training_pl:True,
                             learning_rate_pl:dis_lr}
                # ld is the negative of the discriminator value (smaller the better)
                # _, ld = sess.run([dis_op, discriminator_loss], feed_dict=feed_dict)
                _, ld, true_tr, pred_tr = sess.run([dis_op, discriminator_loss, true_train_label, pred_train_label], feed_dict=feed_dict)
                train_loss_dis += ld
                train_roc += metrics.roc_auc_score(true_tr,pred_tr)

                ### Generator Training ###
                feed_dict = {input_pl: trainx_copy[ran_from:ran_to],
                             is_training_pl:True,
                             learning_rate_pl:gen_lr}
                # lg is the generator's objective (smaller the better)
                _, lg = sess.run([gen_op, generator_loss], feed_dict=feed_dict)
                train_loss_gen += lg

                # update the batch
                train_batch += 1
            
            # get the average loss per batch for both GEN and DIS
            train_loss_gen /= nr_batches_train
            train_loss_gen_log.append(train_loss_gen)
            train_loss_dis /= nr_batches_train
            train_loss_dis_log.append(train_loss_dis)
            train_roc /= nr_batches_train
            train_roc_log.append(train_roc) # len: 50

            ################# TEST of the DISCRIMINATOR #################
            # test discriminator roc
            test_feed_dict = {input_pl:testx,
                              is_training_pl:False}
            pred_te = sess.run(pred_test_label, feed_dict=test_feed_dict)
            test_roc = metrics.roc_auc_score(testy,pred_te) 
            test_roc_log.append(test_roc) # len: nb_epochs

            print("Epoch %d | time = %ds | loss gen = %.4f | loss dis = %.4f | auc train = %.4f | auc test = %.4f"
                  % (epoch, time.time() - begin, train_loss_gen, train_loss_dis, train_roc, test_roc))

            epoch += 1

            size_neg_exp = paras.size_neg_exp()
            
            if (epoch%5 == 0):
                ### Generation Visaulisation ###
                print('Generating examples...')
                z_test_gen = tf.random_normal([size_neg_exp, latent_dim], mean=0.0, stddev=1.0)
                gen_images = gen(z_test_gen, reuse=True) # [50000 x 28 x 28 x 1]
                
                # gen_feed_dict = {input_pl:testx[:25], is_training_pl:False}
                gen_feed_dict = {is_training_pl:False}
     
                gen_exps = sess.run(gen_images, feed_dict = gen_feed_dict)

                gen_exp_log.append(gen_exps)

                # get 25 images to plot
                images = gen_exps[rng.permutation(size_neg_exp)][:25]
                to_plot.append(images)



              
        print('Training terminated')
        
    return to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log
        
        
        
def run(paras,testx,testy):
    """ Runs the training process"""
    print('Running...')
    with tf.Graph().as_default():
        # Set the graph level seed
        tf.set_random_seed(paras.random_seed())
        to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = train_and_test(paras,testx,testy)
    
    return to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log


## SVM set-ip

In [0]:
#@title SVM and post-processing
def postprocess(neg_exps, pos_exps, length=8000, neg_train_ratio=0.25):
    '''
        Args:
            neg_exps: list containing all generated examples from GAN (anomalous)
            pos_exps: np array, the training set, size: 50k (normal)
            length: int, size of neg and pos examples
            neg_train_ratio: float, size of neg examples in training relative to pos examples
    '''
    rng = np.random.RandomState(paras.random_seed())
    train_feat = []
    train_svm_lab = []
    pos_exps = pos_exps.reshape(-1,784)[:length] # [10k x 784]
    neg_length = int(length * neg_train_ratio)

    if neg_length == 0:
        for i in range(10):
            train_feat.append(pos_exps)
            train_svm_lab.append(np.ones(pos_exps.shape[0],dtype=int))
        return train_feat, train_svm_lab
    

    # Produce random noise from uniform (-1,1)  OR  U(0,1)
    random_noise = np.random.uniform(low=-1.0,high=1.0,size=(neg_length,784)) # [10k x 784]
    random_train = np.concatenate((random_noise, pos_exps), axis=0) # [20k x 784]

    # ano: -1, nor: 1
    random_svm_label = np.concatenate((-1 * np.ones(random_noise.shape[0],dtype=int),
                                np.ones(pos_exps.shape[0],dtype=int)), axis=0)

    rand_ind = rng.permutation(random_train.shape[0])
    train_feat.append(random_train[rand_ind])
    train_svm_lab.append(random_svm_label[rand_ind])

    for neg_exp in neg_exps:
        neg_exp = neg_exp.reshape(-1,784)[:neg_length] # [10k x 784]
        concat_feat = np.concatenate((neg_exp, pos_exps), axis=0) # [20k x 784]     
        
        # neg: -1, pos: 1
        concat_svm_lab = np.concatenate((-1 * np.ones(neg_exp.shape[0], dtype=int),
                                np.ones(pos_exps.shape[0], dtype=int)), axis=0) # [20k]
        
        index = rng.permutation(concat_feat.shape[0])
        train_feat.append(concat_feat[index]) # 10 * [20k x 784]
        train_svm_lab.append(concat_svm_lab[index]) # 10 * [20k]

        
    print('Post-processing done')
    
    return train_feat, train_svm_lab

def get_test_valid_svm_dis(anom_noise=False,noise_width=2):
    ano_lab = paras.label()
    testx, tempy = get_test(ano_lab)
    validx, validy = get_valid(ano_lab)
    testx = testx.reshape(-1,784)
    validx = validx.reshape(-1,784)
    testx_border = np.copy(testx)
    testx_uniform = np.copy(testx)
    testx_gauss = np.copy(testx)
    validx_border = np.copy(validx) 
    validx_uniform = np.copy(validx) 
    validx_gauss = np.copy(validx) 

    if ano_lab == 'external':
        testy_dis = ((tempy+1)/2).astype(int)
        testy_dis = testy_dis.astype(int)
        testy_svm = np.copy(tempy)
        validy_svm = np.copy(validy)

        return validx, validy_svm, testx, testy_svm, testy_dis
    
    testy_svm = np.copy(tempy)
    testy_dis = np.copy(tempy)
    validy_svm = np.copy(validy)
    
    # get indices
    anom_index_test = np.isin(tempy,ano_lab)
    norm_index_test = np.invert(anom_index_test)

    anom_index_valid = np.isin(validy,ano_lab)
    norm_index_valid = np.invert(anom_index_valid)

    if anom_noise == True:
        
        noise_width_c = int(28 - noise_width)
        valid_anom_temp = np.copy(validx[anom_index_valid]).reshape(-1,28,28)
        test_anom_temp = np.copy(testx[anom_index_test]).reshape(-1,28,28)
        valid_anom_temp[:,:noise_width,:] += np.random.uniform(low=-1.,high=1.,size=valid_anom_temp[:,:noise_width,:].shape)
        valid_anom_temp[:,:,:noise_width] += np.random.uniform(low=-1.,high=1.,size=valid_anom_temp[:,:,:noise_width].shape)
        valid_anom_temp[:,noise_width_c:,:] += np.random.uniform(low=-1.,high=1.,size=valid_anom_temp[:,noise_width_c:,:].shape)
        valid_anom_temp[:,:,noise_width_c:] += np.random.uniform(low=-1.,high=1.,size=valid_anom_temp[:,:,noise_width_c:].shape)
        test_anom_temp[:,:noise_width,:] += np.random.uniform(low=-1.,high=1.,size=test_anom_temp[:,:noise_width,:].shape)
        test_anom_temp[:,:,:noise_width] += np.random.uniform(low=-1.,high=1.,size=test_anom_temp[:,:,:noise_width].shape)
        test_anom_temp[:,noise_width_c:,:] += np.random.uniform(low=-1.,high=1.,size=test_anom_temp[:,noise_width_c:,:].shape)
        test_anom_temp[:,:,noise_width_c:] += np.random.uniform(low=-1.,high=1.,size=test_anom_temp[:,:,noise_width_c:].shape)
        validx_border[anom_index_valid] = valid_anom_temp.reshape(-1,784)
        testx_border[anom_index_valid] = test_anom_temp.reshape(-1,784)


        valid_anom_temp = np.copy(validx[anom_index_valid]).reshape(-1,28,28)
        test_anom_temp = np.copy(testx[anom_index_test]).reshape(-1,28,28)
        valid_anom_temp += np.random.uniform(low=-1.,high=1.,size=valid_anom_temp.shape) 
        test_anom_temp += np.random.uniform(low=-1.,high=1.,size=test_anom_temp.shape)
        validx_uniform[anom_index_valid] = valid_anom_temp.reshape(-1,784)
        testx_uniform[anom_index_valid] = test_anom_temp.reshape(-1,784)


        valid_anom_temp = np.copy(validx[anom_index_valid]).reshape(-1,28,28)
        test_anom_temp = np.copy(testx[anom_index_test]).reshape(-1,28,28)
        valid_anom_temp += np.random.normal(loc=0.0,scale=0.3,size=valid_anom_temp.shape) 
        test_anom_temp += np.random.normal(loc=0.0,scale=0.3,size=test_anom_temp.shape)
        validx_gauss[anom_index_valid] = valid_anom_temp.reshape(-1,784)
        testx_gauss[anom_index_valid] = test_anom_temp.reshape(-1,784)

    # SVM: -1 for anom, 1 for norm
    testy_svm[anom_index_test] = -1
    testy_svm[norm_index_test] = 1
    validy_svm[anom_index_valid] = -1
    validy_svm[norm_index_valid] = 1
    
    # Discriminator: 0 for anom, 1 for norm
    testy_dis[anom_index_test] = 0
    testy_dis[norm_index_test] = 1
    
    return validx, validx_border, validx_uniform, validx_gauss, validy_svm, testx, testx_border, testx_uniform, testx_gauss, testy_svm, testy_dis

def build_svm(oc=False,_nu=0.5,_gamma='auto',maxiter=-1):
    if oc == False:
        clf = svm.NuSVC(nu=_nu,gamma=_gamma,max_iter=maxiter)
    if oc == True:
        clf = svm.OneClassSVM(nu=_nu,gamma=_gamma,max_iter=maxiter)

    return clf

def plot_25(image):
    '''
        Plot 25 images generated by GEN every 5 epochs
        Args: ?*[25 x 28 x 28 x 1] (? is the integer part of nb_epochs/5)
    '''
    plots = image.reshape(-1,28,28) # [25 x 28 x 28]
    ind = 0
    plt.show()
    while ind < 25:
        plt.subplot(5,5,ind+1)
        img = plt.imshow(plots[ind,:,:])
        plt.axis('off')
        ind += 1

In [0]:
#@title Get valid and test set

validx, validx_border, validx_uniform, validx_gauss, validy_svm, testx, testx_border, testx_uniform, testx_gauss, testy_svm, testy_dis = get_test_valid_svm_dis(anom_noise=True,noise_width=6)
# validx, validy_svm, testx, testy_svm, testy_dis = get_test_valid_svm_dis()

print('Valid shapes: ', validx.shape, validy_svm.shape)
print('Test shapes: ',testx.shape, testy_svm.shape, testy_dis.shape)

## GAN training

In [0]:
#@title training
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
#@title losses plot
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
#@title AuROC plot
plot_roc_curves(train_roc_log, test_roc_log)

In [0]:
#@title examples generated
plot_gen(gen_exp_log)

In [0]:
#@title test AuROC score
roc_score = np.mean(test_roc_log[-5:])
print("Test AuROC: ", roc_score)

## SVM training

In [0]:
#@title Get training set
pos_x, pos_y = get_train(ano_lab)
train_feat, train_svm_lab = postprocess_multi(neg_nm, pos_x, pos_y, length=5000, neg_train_ratio=1.0)
print('Train shapes: ', train_feat[0].shape, train_svm_lab[0].shape)

### Multi-class SVM

In [0]:
#@title Multi-class utilities
def many2two(trainy,validy,testy):
    train_out = []
    for train_many in trainy:
        anom_index_train = np.isin(train_many,-1)
        norm_index_train = np.invert(anom_index_train)
        train_two = np.copy(train_many)
        train_two[norm_index_train] = 1
        train_out.append(train_two)

    anom_index_valid = np.isin(validy,-1)
    norm_index_valid = np.invert(anom_index_valid)   
    valid_two = np.copy(validy)
    valid_two[norm_index_valid] = 1
    
    anom_index_test = np.isin(testy_svm,-1)
    norm_index_test = np.invert(anom_index_test)
    test_two = np.copy(testy)
    test_two[norm_index_test] = 1
    
    return train_out, valid_two, test_two

trainy, validy, testy = many2two(train_svm_lab, validy_svm, testy_svm)

In [0]:
#@title Multi-class SVM
for i in [0,2,4,6,8,10,12,14]:
    print('%d epochs'%5*i)
    iter_ = -1
    for nu_ in [0.1]:
        for gamma_ in [0.01]:
            print('nu: ', nu_)
            print('gamma:', gamma_)
            begin = time.time()
            svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
            x_2c = train_feat[i]
            y_2c = train_svm_lab[i]
            svm_2c.fit(x_2c,y_2c)
            print('training completed')
            train_predy = svm_2c.predict(x_2c)
            valid_predy = svm_2c.predict(validx)
            test_predy = svm_2c.predict(testx)
            print('prediction completed')
            norm_pred_ind_tr = np.invert(np.isin(train_predy,-1))
            norm_pred_ind_va = np.invert(np.isin(valid_predy,-1))
            norm_pred_ind_te = np.invert(np.isin(test_predy,-1))
            train_predy[norm_pred_ind_tr] = 1
            valid_predy[norm_pred_ind_va] = 1
            test_predy[norm_pred_ind_te] = 1
            prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy, test_predy, beta=1.0, pos_label=1, average='binary')
            train_area = metrics.roc_auc_score(trainy[i], train_predy)
            valid_area = metrics.roc_auc_score(validy, valid_predy)
            test_area = metrics.roc_auc_score(testy, test_predy)
            cf_valid = metrics.confusion_matrix(validy, valid_predy)
            cf_test = metrics.confusion_matrix(testy, test_predy)

            print('F1: ', f1)
            print('valid Confusion matrix:\n', cf_valid)
            print('test Confusion matrix:\n', cf_test)
            print('train AUROC: ', train_area)
            print('valid AUROC: ', valid_area)
            print('test AUROC: ', test_area)
            print('time: ', time.time()-begin)
            print('\n\n')

### Two-class SVM

In [0]:
#@title SVM and post-processing
def postprocess_two(neg_exps, pos_exps, length=8000, neg_train_ratio=0.25):
    '''
        Args:
            neg_exps: list containing all generated examples from GAN (anomalous)
            pos_exps: np array, the training set, size: 50k (normal)
            length: int, size of neg and pos examples
            neg_train_ratio: float, size of neg examples in training relative to pos examples
    '''
    rng = np.random.RandomState(None)
    train_feat = []
    train_svm_lab = []
    pos_exps = pos_exps.reshape(-1,784)[:length] # [10k x 784]
    neg_length = int(length * neg_train_ratio)

    if neg_length == 0:
        for i in range(10):
            train_feat.append(pos_exps)
            train_svm_lab.append(np.ones(pos_exps.shape[0],dtype=int))
        return train_feat, train_svm_lab
    

    # Produce random noise from uniform (-1,1)  OR  U(0,1)
    random_noise = np.random.uniform(low=-1.0,high=1.0,size=(neg_length,784)) # [10k x 784]
    random_train = np.concatenate((random_noise, pos_exps), axis=0) # [20k x 784]

    # ano: -1, nor: 1
    random_svm_label = np.concatenate((-1 * np.ones(random_noise.shape[0],dtype=int),
                                np.ones(pos_exps.shape[0],dtype=int)), axis=0)

    rand_ind = rng.permutation(random_train.shape[0])
    train_feat.append(random_train[rand_ind])
    train_svm_lab.append(random_svm_label[rand_ind])

    for neg_exp in neg_exps:
        neg_exp = neg_exp.reshape(-1,784)[:neg_length] # [10k x 784]
        concat_feat = np.concatenate((neg_exp, pos_exps), axis=0) # [20k x 784]     
        
        # neg: -1, pos: 1
        concat_svm_lab = np.concatenate((-1 * np.ones(neg_exp.shape[0], dtype=int),
                                np.ones(pos_exps.shape[0], dtype=int)), axis=0) # [20k]
        
        index = rng.permutation(concat_feat.shape[0])
        train_feat.append(concat_feat[index]) # 10 * [20k x 784]
        train_svm_lab.append(concat_svm_lab[index]) # 10 * [20k]

        
    print('Post-processing done')
    
    return train_feat, train_svm_lab

def get_test_valid_svm_two(ano_lab):
    ano_lab = ano_lab
    testx, tempy = get_test(ano_lab)
    validx, validy = get_valid(ano_lab)
    testx = testx.reshape(-1,784)
    validx = validx.reshape(-1,784)

    if ano_lab == 'external':
        testy_svm = np.copy(tempy)
        validy_svm = np.copy(validy)

        return validx, validy_svm, testx, testy_svm
    
    testy_svm = np.copy(tempy)
    validy_svm = np.copy(validy)
    
    # get indices
    anom_index_test = np.isin(tempy,ano_lab)
    norm_index_test = np.invert(anom_index_test)

    anom_index_valid = np.isin(validy,ano_lab)
    norm_index_valid = np.invert(anom_index_valid)

    # SVM: -1 for anom, 1 for norm
    testy_svm[anom_index_test] = -1
    testy_svm[norm_index_test] = 1
    validy_svm[anom_index_valid] = -1
    validy_svm[norm_index_valid] = 1
    
    
    return validx, validy_svm, testx, testy_svm

def build_svm(oc=False,_nu=0.5,_gamma='auto',maxiter=-1):
    if oc == False:
        clf = svm.NuSVC(nu=_nu,gamma=_gamma,max_iter=maxiter)
    if oc == True:
        clf = svm.OneClassSVM(nu=_nu,gamma=_gamma,max_iter=maxiter)

    return clf

def plot_25(image):
    '''
        Plot 25 images generated by GEN every 5 epochs
        Args: ?*[25 x 28 x 28 x 1] (? is the integer part of nb_epochs/5)
    '''
    plots = image.reshape(-1,28,28) # [25 x 28 x 28]
    ind = 0
    plt.show()
    while ind < 25:
        plt.subplot(5,5,ind+1)
        img = plt.imshow(plots[ind,:,:])
        plt.axis('off')
        ind += 1

In [0]:
#@title Two-class SVM
for i in [0,2,4,6,8,10,12,14]:
    print('%d epochs'%5*i)
    iter_ = -1
    for nu_ in [0.01,0.03,0.05,0.07,0.09]:
        for gamma_ in [1e-6,1e-5,1e-4,1e-3,1e-2]:
            begin = time.time()
            svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
            x_2c = train_feat[i]
            y_2c = train_svm_lab[i]
            svm_2c.fit(x_2c,y_2c)
            print('training completed')
            train_predy = svm_2c.predict(x_2c)
            valid_predy = svm_2c.predict(validx)
            test_predy = svm_2c.predict(testx)
            print('prediction completed')
            prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, test_predy, beta=1.0, pos_label=1, average='binary')
            train_area = metrics.roc_auc_score(y_2c, train_predy)
            valid_area = metrics.roc_auc_score(validy_svm, valid_predy)
            test_area = metrics.roc_auc_score(testy_svm, test_predy)
            cf_valid = metrics.confusion_matrix(validy_svm, valid_predy)
            cf_test = metrics.confusion_matrix(testy_svm, test_predy)
            print('nu: ', nu_)
            print('gamma:', gamma_)
            print('iter: ', iter_)
            print('F1: ', f1)
            print('Precision: ', prec)
            print('Recall: ', reca)
            print('valid Confusion matrix:\n', cf_valid)
            print('test Confusion matrix:\n', cf_test)
            print('train AUROC: ', train_area)
            print('valid AUROC: ', valid_area)
            print('test AUROC: ', test_area)
            print('time: ', time.time()-begin)
            print('\n\n')

# Legacy

### SVMs

In [0]:
pos_x,_ = get_train(paras.label())
train_feat_4q, train_svm_lab_4q = postprocess(gen_exp_log, pos_x, length=5000, neg_train_ratio=1.0)
print('Train shapes: ', train_feat_4q[0].shape, train_svm_lab_4q[0].shape)

In [0]:
print('70 epochs')
iter_ = -1
for nu_ in [0.01,0.03,0.05,0.07,0.09]:
# for nu_ in [1e-3,5e-3,0.01]:

    for gamma_ in [1e-6,1e-5,1e-4,1e-3,1e-2]:
#     for gamma_ in [1e-5,2e-5,3e-5,4e-5,5e-5,6e-5,7e-5,8e-5,1e-4]:
    #     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    #     for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_4q[-1]
        y_2c = train_svm_lab_4q[-1]
        svm_2c.fit(x_2c,y_2c)
        print('training completed')
        train_predy = svm_2c.predict(x_2c)
    #     valid_predy_bias = new_predict(svm_2c, -100, validx)
        valid_predy = svm_2c.predict(validx)
        test_predy = svm_2c.predict(testx)
        print('prediction completed')
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, test_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)
        test_area = metrics.roc_auc_score(testy_svm, test_predy)
        #     bias_area = metrics.roc_auc_score(validy_svm, valid_predy_bias)
        cf_valid = metrics.confusion_matrix(validy_svm, valid_predy)
        cf_test = metrics.confusion_matrix(testy_svm, test_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf_valid)
        print('test Confusion matrix:\n', cf_test)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('test AUROC: ', test_area)
        #     print('biased AUROC: ', bias_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('70 epochs')
iter_ = -1
for nu_ in [0.01,0.03,0.05,0.07,0.09]:
# for nu_ in [1e-3,5e-3,0.01]:

#     for gamma_ in [1e-6,1e-5,1e-4,1e-3,1e-2]:
    for gamma_ in [5e-6,3e-5,5e-5,7e-5]:
    #     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    #     for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_4q[-1]
        y_2c = train_svm_lab_4q[-1]
        svm_2c.fit(x_2c,y_2c)
        print('training completed')
        train_predy = svm_2c.predict(x_2c)
    #     valid_predy_bias = new_predict(svm_2c, -100, validx)
        valid_predy = svm_2c.predict(validx)
        test_predy = svm_2c.predict(testx)
        print('prediction completed')
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, test_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)
        test_area = metrics.roc_auc_score(testy_svm, test_predy)
        #     bias_area = metrics.roc_auc_score(validy_svm, valid_predy_bias)
        cf_valid = metrics.confusion_matrix(validy_svm, valid_predy)
        cf_test = metrics.confusion_matrix(testy_svm, test_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf_valid)
        print('test Confusion matrix:\n', cf_test)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('test AUROC: ', test_area)
        #     print('biased AUROC: ', bias_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('70 epochs')
iter_ = -1
for nu_ in [0.01,0.03,0.05,0.07,0.09]:
# for nu_ in [1e-3,5e-3,0.01]:

    for gamma_ in [1e-6,1e-5,1e-4,1e-3,1e-2]:
#     for gamma_ in [1e-5,2e-5,3e-5,4e-5,5e-5,6e-5,7e-5,8e-5,1e-4]:
    #     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    #     for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_4q[12]
        y_2c = train_svm_lab_4q[12]
        svm_2c.fit(x_2c,y_2c)
        print('training completed')
        train_predy = svm_2c.predict(x_2c)
    #     valid_predy_bias = new_predict(svm_2c, -100, validx)
        valid_predy = svm_2c.predict(validx)
        test_predy = svm_2c.predict(testx)
        print('prediction completed')
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, test_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)
        test_area = metrics.roc_auc_score(testy_svm, test_predy)
        #     bias_area = metrics.roc_auc_score(validy_svm, valid_predy_bias)
        cf_valid = metrics.confusion_matrix(validy_svm, valid_predy)
        cf_test = metrics.confusion_matrix(testy_svm, test_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf_valid)
        print('test Confusion matrix:\n', cf_test)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('test AUROC: ', test_area)
        #     print('biased AUROC: ', bias_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('70 epochs')
iter_ = -1
nu_ = 7e-2
gamma_ = 1e-5
# for nu_ in [0.01,0.03,0.05,0.07,0.09]:
# for nu_ in [1e-3,5e-3,0.01]:

#     for gamma_ in [1e-6,1e-5,1e-4,1e-3,1e-2]:
#     for gamma_ in [1e-5,2e-5,3e-5,4e-5,5e-5,6e-5,7e-5,8e-5,1e-4]:
    #     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    #     for nu_ in [5e-2]:
begin = time.time()
# svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
shoot = svm.NuSVC(nu=nu_,gamma=gamma_,class_weight={1: 1,-1: 10000})
x_2c = train_feat_4q[12]
y_2c = train_svm_lab_4q[12]
shoot.fit(x_2c,y_2c)
print('training completed')
train_predy = shoot.predict(x_2c)
#     valid_predy_bias = new_predict(svm_2c, -100, validx)
valid_predy = shoot.predict(validx)
test_predy = shoot.predict(testx)
print('prediction completed')
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, test_predy, beta=1.0, pos_label=1, average='binary')
train_area = metrics.roc_auc_score(y_2c, train_predy)
valid_area = metrics.roc_auc_score(validy_svm, valid_predy)
test_area = metrics.roc_auc_score(testy_svm, test_predy)
#     bias_area = metrics.roc_auc_score(validy_svm, valid_predy_bias)
cf_valid = metrics.confusion_matrix(validy_svm, valid_predy)
cf_test = metrics.confusion_matrix(testy_svm, test_predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('valid Confusion matrix:\n', cf_valid)
print('test Confusion matrix:\n', cf_test)
print('train AUROC: ', train_area)
print('valid AUROC: ', valid_area)
print('test AUROC: ', test_area)
#     print('biased AUROC: ', bias_area)
print('time: ', time.time()-begin)
print('\n\n')

## Ano = (3,5)

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_gen(gen_exp_log)

### Take 2

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

In [0]:
np.mean(test_roc_log[-5:])

### Take 1 | 0.754

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

In [0]:
np.mean(test_roc_log[-5:])

## Ano = 0

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

In [0]:
np.mean(test_roc_log[-4:])

In [0]:
pos_x,_ = get_train(paras.label())
train_feat_4q, train_svm_lab_4q = postprocess(gen_exp_log, pos_x, length=20000, neg_train_ratio=1.0)
print('Train shapes: ', train_feat_4q[0].shape, train_svm_lab_4q[0].shape)

In [0]:
print('70 epochs')
iter_ = -1
# for nu_ in [0.01,0.03,0.05,0.07,0.09]:
for nu_ in [1e-3,5e-3,0.01]:

#     for gamma_ in [1e-6,1e-5,1e-4,1e-3,1e-2]:
    for gamma_ in [1e-5,2e-5,3e-5,4e-5,5e-5,6e-5,7e-5,8e-5,1e-4]:
    #     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    #     for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_4q[-1]
        y_2c = train_svm_lab_4q[-1]
        svm_2c.fit(x_2c,y_2c)
        print('training completed')
        train_predy = svm_2c.predict(x_2c)
    #     valid_predy_bias = new_predict(svm_2c, -100, validx)
        valid_predy = svm_2c.predict(validx)
        test_predy = svm_2c.predict(testx)
        print('prediction completed')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)
        test_area = metrics.roc_auc_score(testy_svm, test_predy)
        #     bias_area = metrics.roc_auc_score(validy_svm, valid_predy_bias)
        cf_valid = metrics.confusion_matrix(validy_svm, valid_predy)
        cf_test = metrics.confusion_matrix(testy_svm, test_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf_valid)
        print('test Confusion matrix:\n', cf_test)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('test AUROC: ', test_area)
        #     print('biased AUROC: ', bias_area)
        print('time: ', time.time()-begin)
        print('\n\n')

## Ano = 1

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

## Ano = 2

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

### Take 2 | 0.781

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

In [0]:
np.mean(test_roc_log[-5:])

### Take 1 | 0.759

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

In [0]:
np.mean(test_roc_log[-5:])

## Ano = 3

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

## Ano = 4

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

## Ano = 5

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

## Ano = 6

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

## Ano = 7

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

## Ano = 8

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

## Ano = 9

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

## OLD 

### Instructions:


1.   Add uniform noise to the broader of anomalies (when after 5 epochs)
2.   Add Gaussian noise to anomalies (when after 50 epochs)
2.   Change to linear kernel



## Ano = 8

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

## Curves

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

## SVM

In [0]:
pos_x,_ = get_train(paras.label())
train_feat_0q, train_svm_lab_0q = postprocess(gen_exp_log, pos_x, length=5000, neg_train_ratio=0)
print('Train shapes: ', train_feat_0q[0].shape, train_svm_lab_0q[0].shape)

In [0]:
pos_x,_ = get_train(paras.label())
train_feat_1q, train_svm_lab_1q = postprocess(gen_exp_log, pos_x, length=4000, neg_train_ratio=0.25)
print('Train shapes: ', train_feat_1q[0].shape, train_svm_lab_1q[0].shape)

In [0]:
pos_x,_ = get_train(paras.label())
train_feat_2q, train_svm_lab_2q = postprocess(gen_exp_log, pos_x, length=3333, neg_train_ratio=0.50)
print('Train shapes: ', train_feat_2q[0].shape, train_svm_lab_2q[0].shape)

In [0]:
pos_x,_ = get_train(paras.label())
train_feat_3q, train_svm_lab_3q = postprocess(gen_exp_log, pos_x, length=2857, neg_train_ratio=0.75)
print('Train shapes: ', train_feat_3q[0].shape, train_svm_lab_3q[0].shape)

In [0]:
pos_x,_ = get_train(paras.label())
train_feat_4q, train_svm_lab_4q = postprocess(gen_exp_log, pos_x, length=2500, neg_train_ratio=1.00)
print('Train shapes: ', train_feat_4q[0].shape, train_svm_lab_4q[0].shape)

## SVMs with changing threshold (b)

In [0]:
def new_predict(clf, new_bias, X):
    print('Old b: ', clf.intercept_)
    print('New b: ', new_bias)
    return np.sign(clf.decision_function(X) - clf.intercept_ + new_bias)

In [0]:
svm_2c.decision_function(x_2c)

In [0]:
x_2c.max()

In [0]:
print('0 epochs')
iter_ = -1
nu_ = 0.01
gamma_=0.05
# for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
#     for nu_ in [5e-2]:
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat_1q[0]
y_2c = train_svm_lab_1q[0]
svm_2c.fit(x_2c,y_2c)
print('training completed')
train_predy = svm_2c.predict(x_2c)
valid_predy_bias = new_predict(svm_2c, -100, validx)
valid_predy = svm_2c.predict(validx)
f1 = metrics.f1_score(validy_svm, valid_predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
train_area = metrics.roc_auc_score(y_2c, train_predy)
valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
bias_area = metrics.roc_auc_score(validy_svm, valid_predy_bias)
cf = metrics.confusion_matrix(validy_svm, valid_predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('valid Confusion matrix:\n', cf)
print('train AUROC: ', train_area)
print('valid AUROC: ', valid_area)
print('biased AUROC: ', bias_area)
print('time: ', time.time()-begin)
print('\n\n')

## Different pos:neg

### 1:0.25

In [0]:
print('0 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_1q[0]
        y_2c = train_svm_lab_1q[0]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('10 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_1q[2]
        y_2c = train_svm_lab_1q[2]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('20 epochs')
iter_ = -1
nu_ = 0.01
# for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
for gamma_ in [8e-5,1e-4,5e-4,1e-3]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_1q[4]
        y_2c = train_svm_lab_1q[4]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('30 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_1q[6]
        y_2c = train_svm_lab_1q[6]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('40 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_1q[8]
        y_2c = train_svm_lab_1q[8]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('50 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_1q[-1]
        y_2c = train_svm_lab_1q[-1]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

### 1:0.5

In [0]:
print('0 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_2q[0]
        y_2c = train_svm_lab_2q[0]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('10 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_2q[2]
        y_2c = train_svm_lab_2q[2]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('20 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_2q[4]
        y_2c = train_svm_lab_2q[4]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('30 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_2q[6]
        y_2c = train_svm_lab_2q[6]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('40 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_2q[8]
        y_2c = train_svm_lab_2q[8]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('50 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_2q[-1]
        y_2c = train_svm_lab_2q[-1]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

### 1:0.75

In [0]:
print('0 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_3q[0]
        y_2c = train_svm_lab_3q[0]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('10 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_3q[2]
        y_2c = train_svm_lab_3q[2]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('20 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_3q[4]
        y_2c = train_svm_lab_3q[4]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('30 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_3q[6]
        y_2c = train_svm_lab_3q[6]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('40 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_3q[8]
        y_2c = train_svm_lab_3q[8]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('50 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_3q[-1]
        y_2c = train_svm_lab_3q[-1]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

### 1:1

In [0]:
print('0 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_4q[0]
        y_2c = train_svm_lab_4q[0]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('10 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_4q[2]
        y_2c = train_svm_lab_4q[2]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('20 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_4q[4]
        y_2c = train_svm_lab_4q[4]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('30 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_4q[6]
        y_2c = train_svm_lab_4q[6]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('40 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_4q[8]
        y_2c = train_svm_lab_4q[8]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
print('50 epochs')
iter_ = -1
nu_ = 0.01
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat_4q[-1]
        y_2c = train_svm_lab_4q[-1]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, valid_predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

# OLD Legacy

## Ano = 8

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

In [0]:
pos_x,_ = get_train(paras.label())
train_feat, train_svm_lab = postprocess(gen_exp_log, pos_x, length=5000, neg_train_ratio=0.1)
print('Train shapes: ', train_feat[0].shape, train_svm_lab[0].shape)

In [0]:
validx, validx_border, validx_uniform, validx_gauss, validy_svm, testx, testx_border, testx_uniform, testx_gauss, testy_svm, testy_dis, valid_anom_temp= get_test_valid_svm_dis(anom_noise=True,noise_width=8)
print('Valid shapes: ', validx.shape, validy_svm.shape)
print('Test shapes: ',testx.shape, testy_svm.shape, testy_dis.shape)

In [0]:
plot_25(train_feat[0][:25])

In [0]:
plot_25(validx[-25:])
plot_25(validx_border[-25:])
plot_25(validx_uniform[-25:])
plot_25(validx_gauss[-25:])

In [0]:
iter_ = -1
nu_ = 0.01
print('50 epochs')
for gamma_ in [5e-5,6e-5,7e-5,8e-5,1e-4]:
#     for nu_ in [1e-2,3e-2,5e-2,7e-2,9e-2]:
    for nu_ in [5e-2]:
        begin = time.time()
        svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
        x_2c = train_feat[-1][:1000]
        y_2c = train_svm_lab[-1][:1000]
        svm_2c.fit(x_2c,y_2c)
        train_predy = svm_2c.predict(x_2c)
        valid_predy = svm_2c.predict(validx)
        f1 = metrics.f1_score(validy_svm, predy)
        prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
        train_area = metrics.roc_auc_score(y_2c, train_predy)
        valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
        cf = metrics.confusion_matrix(validy_svm, valid_predy)
        print('nu: ', nu_)
        print('gamma:', gamma_)
        print('iter: ', iter_)
        print('F1: ', f1)
        print('Precision: ', prec)
        print('Recall: ', reca)
        print('valid Confusion matrix:\n', cf)
        print('train AUROC: ', train_area)
        print('valid AUROC: ', valid_area)
        print('time: ', time.time()-begin)
        print('\n\n')

In [0]:
iter_ = -1
nu_ = 0.01
gamma_ = 'auto'
print('50 epochs')
for nu_ in [1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,1e-1]:
    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[-1]
    y_2c = train_svm_lab[-1]
    svm_2c.fit(x_2c,y_2c)
    train_predy = svm_2c.predict(x_2c)
    valid_predy = svm_2c.predict(validx)
    f1 = metrics.f1_score(validy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, valid_predy, beta=1.0, pos_label=1, average='binary')
    train_area = metrics.roc_auc_score(y_2c, train_predy)
    valid_area = metrics.roc_auc_score(validy_svm, valid_predy)  
    cf = metrics.confusion_matrix(validy_svm, valid_predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('valid Confusion matrix:\n', cf)
    print('train AUROC: ', train_area)
    print('valid AUROC: ', valid_area)
    print('time: ', time.time()-begin)
    print('\n\n')

In [0]:
iter_ = -1
nu_=0.01
print('50 epochs')
# for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:
for gamma_ in [1e-2]:
    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[0]
    y_2c = train_svm_lab[0]
    svm_2c.fit(x_2c,y_2c)
    predy = svm_2c.predict(validx_temp)
    f1 = metrics.f1_score(validy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(validy_svm, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(validy_svm, predy)  
    cf = metrics.confusion_matrix(validy_svm, predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('Confusion matrix:\n', cf)
    print('time: ', time.time()-begin)
    print('\n\n')

## AuROC of DIS (train and test) | anom='external' | 50 epochs

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

In [0]:
pos_x,_ = get_train(paras.label())
train_feat, train_svm_lab = postprocess(gen_exp_log, pos_x, length=8000, neg_train_ratio=0.25)
print('Train shapes: ', train_feat[0].shape, train_svm_lab[0].shape)

In [0]:
nu_ = 0.05
gamma_ = 5e-4
iter_= -1
# for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:

In [0]:
nu_ = 0.01
print('50 epochs')
for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:

    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[-1]
    y_2c = train_svm_lab[-1]
    svm_2c.fit(x_2c,y_2c)
    predy = svm_2c.predict(testx)
    f1 = metrics.f1_score(testy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy_svm, predy)  
    cf = metrics.confusion_matrix(testy_svm, predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('Confusion matrix:\n', cf)
    print('time: ', time.time()-begin)
    print('\n\n')

## AuROC of DIS (train and test) | anom=8 | 200 epochs

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log, train_roc_log, test_roc_log = run(paras,testx,testy_dis)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
plot_roc_curves(train_roc_log, test_roc_log)

In [0]:
pos_x,_ = get_train(paras.label())
train_feat, train_svm_lab = postprocess(gen_exp_log, pos_x, length=8000, neg_train_ratio=0.25)
print('Train shapes: ', train_feat[0].shape, train_svm_lab[0].shape)

### SVMs | anom=8 | training: {8000 pos and 2000 neg} | test: {4000 pos and 1000 neg}### SVMs | training:

In [0]:
nu_ = 0.05
gamma_ = 5e-4
iter_= -1
# for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:

In [0]:
nu_ = 0.01
print('50 epochs')
for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:

    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[-1]
    y_2c = train_svm_lab[-1]
    svm_2c.fit(x_2c,y_2c)
    predy = svm_2c.predict(testx)
    f1 = metrics.f1_score(testy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy_svm, predy)  
    cf = metrics.confusion_matrix(testy_svm, predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('Confusion matrix:\n', cf)
    print('time: ', time.time()-begin)

## SVMs | anom=8 | training: {10000 pos and 10000 neg} | test: {4000 pos and 1000 neg}

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log  = run(paras)

In [0]:
plot_gen(to_plot)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
pos_x,_ = get_train(paras.label())
train_feat, train_svm_lab = postprocess(gen_exp_log, pos_x)
testx, testy_svm = get_test_svm_cnn()
print('Train shapes: ', train_feat[0].shape, train_svm_lab[0].shape)
print('Test shapes: ',testx.shape,testy_svm.shape)

In [0]:
nu_ = 0.05
gamma_ = 5e-4
iter_= -1
# for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:


In [0]:
nu_ = 0.01
print('50 epochs')
for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:

    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[-1]
    y_2c = train_svm_lab[-1]
    svm_2c.fit(x_2c,y_2c)
    predy = svm_2c.predict(testx)
    f1 = metrics.f1_score(testy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy_svm, predy)  
    cf = metrics.confusion_matrix(testy_svm, predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('Confusion matrix:\n', cf)
    print('time: ', time.time()-begin)

In [0]:
nu_=0.01
print('40 epochs')
for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:
    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[8]
    y_2c = train_svm_lab[8]
    svm_2c.fit(x_2c,y_2c)
    predy = svm_2c.predict(testx)
    f1 = metrics.f1_score(testy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy_svm, predy)  
    cf = metrics.confusion_matrix(testy_svm, predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('Confusion matrix:\n', cf)
    print('time: ', time.time()-begin)

In [0]:
nu_=0.01
print('30 epochs')
for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:
    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[6]
    y_2c = train_svm_lab[6]
    svm_2c.fit(x_2c,y_2c)
    predy = svm_2c.predict(testx)
    f1 = metrics.f1_score(testy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy_svm, predy)  
    cf = metrics.confusion_matrix(testy_svm, predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('Confusion matrix:\n', cf)
    print('time: ', time.time()-begin)

In [0]:
nu_ = 0.05
gamma_ = 3e-4
iter_= -1
# for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:


In [0]:
print('40 epochs')
for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:

begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[8]
y_2c = train_svm_lab[8]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
print('30 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[6]
y_2c = train_svm_lab[6]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
print('20 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[4]
y_2c = train_svm_lab[4]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
print('10 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[2]
y_2c = train_svm_lab[2]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
print('0 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[0]
y_2c = train_svm_lab[0]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
nu_ = 0.01
gamma_ = 5e-4
iter_= -1
# for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:


In [0]:
print('50 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[-1]
y_2c = train_svm_lab[-1]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
print('40 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[8]
y_2c = train_svm_lab[8]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
print('30 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[6]
y_2c = train_svm_lab[6]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
print('20 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[4]
y_2c = train_svm_lab[4]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
print('10 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[2]
y_2c = train_svm_lab[2]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
print('0 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[0]
y_2c = train_svm_lab[0]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
nu_=0.1
print('50 epochs')
for gamma_ in [1e-5,1e-4,1e-3,1e-2,0.1,1,10,100]:
    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[-1]
    y_2c = train_svm_lab[-1]
    svm_2c.fit(x_2c,y_2c)
    predy = svm_2c.predict(testx)
    f1 = metrics.f1_score(testy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy_svm, predy)  
    cf = metrics.confusion_matrix(testy_svm, predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('Confusion matrix:\n', cf)
    print('time: ', time.time()-begin)

In [0]:
nu_=0.05
print('50 epochs')
for gamma_ in [1e-4,3e-4,5e-4,7e-4,8e-4,1e-3,5e-3,1e-2]:
    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[-1]
    y_2c = train_svm_lab[-1]
    svm_2c.fit(x_2c,y_2c)
    predy = svm_2c.predict(testx)
    f1 = metrics.f1_score(testy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy_svm, predy)  
    cf = metrics.confusion_matrix(testy_svm, predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('Confusion matrix:\n', cf)
    print('time: ', time.time()-begin)

In [0]:
nu_=0.03
print('50 epochs')
for gamma_ in [1e-5,1e-4,1e-3,1e-2,0.1,1,10,100]:
    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[-1]
    y_2c = train_svm_lab[-1]
    svm_2c.fit(x_2c,y_2c)
    predy = svm_2c.predict(testx)
    f1 = metrics.f1_score(testy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy_svm, predy)  
    cf = metrics.confusion_matrix(testy_svm, predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('Confusion matrix:\n', cf)
    print('time: ', time.time()-begin)

In [0]:
nu_=0.07
print('50 epochs')
for gamma_ in [1e-5,1e-4,5e-4,1e-3,5e-3,1e-2,0.1]:
    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[-1]
    y_2c = train_svm_lab[-1]
    svm_2c.fit(x_2c,y_2c)
    predy = svm_2c.predict(testx)
    f1 = metrics.f1_score(testy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy_svm, predy)  
    cf = metrics.confusion_matrix(testy_svm, predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('Confusion matrix:\n', cf)
    print('time: ', time.time()-begin)

In [0]:
nu_=0.5
print('50 epochs')
for gamma_ in [1e-5,1e-4,5e-4,1e-3,5e-3,1e-2,0.1]:
    begin = time.time()
    svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
    x_2c = train_feat[-1]
    y_2c = train_svm_lab[-1]
    svm_2c.fit(x_2c,y_2c)
    predy = svm_2c.predict(testx)
    f1 = metrics.f1_score(testy_svm, predy)
    prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
    area = metrics.roc_auc_score(testy_svm, predy)  
    cf = metrics.confusion_matrix(testy_svm, predy)
    print('nu: ', nu_)
    print('gamma:', gamma_)
    print('iter: ', iter_)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', reca)
    print('area under roc: ', area)
    print('Confusion matrix:\n', cf)
    print('time: ', time.time()-begin)

## Ano = 8,  centred = False

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log  = run(paras)

In [0]:
plot_gen(to_plot)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
pos_x,_ = get_train(paras.label())
train_feat, train_svm_lab = postprocess(gen_exp_log, pos_x)
testx, testy_svm = get_test_svm_cnn()
print('Train shapes: ', train_feat[0].shape, train_svm_lab[0].shape)
print('Test shapes: ',testx.shape,testy_svm.shape)

In [0]:
nu_ = 0.01
gamma_ = 1e-5
iter_=10

In [0]:
print('50 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[-1]
y_2c = train_svm_lab[-1]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
print('50 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[-1]
y_2c = train_svm_lab[-1]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

In [0]:
print('50 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[-1]
y_2c = train_svm_lab[-1]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('gamma:', gamma_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:\n', cf)
print('time: ', time.time()-begin)

## Ano = 8,  centred = True

In [0]:
to_plot, train_loss_gen_log, train_loss_dis_log, gen_exp_log  = run(paras)

In [0]:
plot_gen(to_plot)

In [0]:
plot_learning_curves(train_loss_gen_log, train_loss_dis_log)

In [0]:
pos_x,_ = get_train(paras.label())
train_feat, train_svm_lab = postprocess(gen_exp_log, pos_x)
testx, testy_svm = get_test_svm_cnn()
print('Train shapes: ', train_feat[0].shape, train_svm_lab[0].shape)
print('Test shapes: ',testx.shape,testy_svm.shape)

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
scl = MinMaxScaler()
new_train = scl.fit_transform(train_feat[-1])
new_test = scl.fit_transform(testx)

In [0]:
plot_25(new_test)

In [0]:
nu_ = 0.01
gamma_ = 1e-6
iter_=10

In [0]:
print('50 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[-1]
y_2c = train_svm_lab[-1]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
cf = metrics.confusion_matrix(testy_svm, predy)
print('nu: ', nu_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('Confusion matrix:'\n, cf)
print('time: ', time.time()-begin)

In [0]:
nu_= 0.01
iter_=-1
gamma_=0.05

In [0]:
print('0 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=iter_)
x_2c = train_feat[0]
y_2c = train_svm_lab[0]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
print('nu: ', nu_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('time: ', time.time()-begin)

In [0]:
print('50 epochs')
begin = time.time()
svm_2c = build_svm(oc=False,_nu=nu_,_gamma=gamma_,maxiter=100)
x_2c = train_feat[-1]
y_2c = train_svm_lab[-1]
svm_2c.fit(x_2c,y_2c)
predy = svm_2c.predict(testx)
f1 = metrics.f1_score(testy_svm, predy)
prec, reca, f1, _ = metrics.precision_recall_fscore_support(testy_svm, predy, beta=1.0, pos_label=1, average='binary')
area = metrics.roc_auc_score(testy_svm, predy)  
print('nu: ', nu_)
print('iter: ', iter_)
print('F1: ', f1)
print('Precision: ', prec)
print('Recall: ', reca)
print('area under roc: ', area)
print('time: ', time.time()-begin)